# Test Qlora

based on [Medium Article](https://medium.com/p/13a7d3a256f6)

In [1]:
# Set those token to clone repo and access huggingface
GIT_TOKEN = ''
HUGGINGFACE_TOKEN = ''

repo = 'SwissTextSDG'
!rm -r SwissTextSDG/
!git clone https://{GIT_TOKEN}@github.com/vera-bernhard/{repo}


rm: cannot remove 'SwissTextSDG/': No such file or directory
Cloning into 'SwissTextSDG'...
remote: Enumerating objects: 1158, done.
remote: Counting objects: 100% (396/396), done.
remote: Compressing objects: 100% (258/258), done.
^Cceiving objects:   8% (93/1158), 90.29 MiB | 22.03 MiB/s


In [1]:
%cd /content/SwissTextSDG/
!git checkout qlora

[Errno 2] No such file or directory: '/content/SwissTextSDG/'
/mnt/home/veralarabernhard/SwissTextSDG/src/models/qlora
M	src/data/dataset.py
M	src/models/qlora/test_qlora.ipynb
Already on 'qlora'
Your branch is up to date with 'origin/qlora'.


In [1]:
!pip install transformers
!pip install accelerate
!pip install peft
!pip install datasets
!pip install bitsandbytes
!pip install swifter
!pip install pytorch_transformers
!pip install huggingface_hub

from config import *
from huggingface_hub import login
login(HUGGINGFACE_TOKEN)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /mnt/home/veralarabernhard/.cache/huggingface/token
Login successful


In [2]:
import torch
from transformers import (AutoTokenizer,
                          MistralForSequenceClassification, 
                          BitsAndBytesConfig, 
                          Trainer, 
                          TrainingArguments)
from datasets import Dataset
from peft import (LoraConfig, 
                  PeftConfig, 
                  PeftModel, 
                  get_peft_model,
                  prepare_model_for_kbit_training)
from sklearn.metrics import f1_score

In [4]:
model_checkpoint = 'mistralai/Mistral-7B-v0.1'

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)

In [6]:
model = MistralForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=18,
        quantization_config=bnb_config,
        device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=2,
    bias='none',
    task_type='SEQ_CLS',
    target_modules=['q_proj', 'v_proj']
)
model = get_peft_model(model, peft_config)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
%cd ../../..

In [17]:
import sys
import os

from src.data.dataset import TrainSwissTextDataset
     

dataset = TrainSwissTextDataset.create_instance(dataset='swisstext_task1_train', model_name='qlora-mistral', use_val=True, train_frac=0.7)
train_df, test_df, val_df = dataset.get_train_test_val()
len(train_df)

142

In [19]:
train_df['text'] = train_df['TITLE'] + ' ' + train_df['ABSTRACT']
val_df['text'] = val_df['TITLE'] + ' ' + val_df['ABSTRACT']
# rename sgd to label
train_df = train_df.rename(columns={'sdg': 'labels'})
val_df = val_df.rename(columns={'sdg': 'labels'})

train_ds = Dataset.from_pandas(train_df, split='train')
val_ds = Dataset.from_pandas(val_df, split='val')


train_ds_tokenized = train_ds.map(tokenize, batched=True)
val_ds_tokenized = val_ds.map(tokenize, batched=True)

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

In [20]:
set(train_df.labels.to_list())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}

In [21]:
def compute_metrics(pred):    
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_result = f1_score(labels, preds, average='weighted')
    return {'f1_score': f1_result}

In [23]:
training_args = TrainingArguments(
    output_dir='./checkpoints',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tokenized,
    tokenizer=tokenizer,
    eval_dataset=val_ds_tokenized,
    compute_metrics=compute_metrics,
)
trainer.train()

[2024-04-28 22:25:00][WARNING]	Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/mnt/home/veralarabernhard/SwissTextSDG/sdg_env/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score
1,No log,14.012147,0.034943
2,No log,13.865309,0.065652
3,No log,13.807353,0.078634
4,5.137700,13.708877,0.078634
5,5.137700,13.696450,0.078634


/mnt/home/veralarabernhard/SwissTextSDG/sdg_env/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/mnt/home/veralarabernhard/SwissTextSDG/sdg_env/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/mnt/home/

TrainOutput(global_step=710, training_loss=3.8956819990990867, metrics={'train_runtime': 5316.1555, 'train_samples_per_second': 0.134, 'train_steps_per_second': 0.134, 'total_flos': 8907788122521600.0, 'train_loss': 3.8956819990990867, 'epoch': 5.0})

In [15]:
model_checkpoint = 'mistralai/Mistral-7B-v0.1'
model = MistralForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=18,
        quantization_config=bnb_config,
        device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
adapter_checkpoint = "/mnt/home/veralarabernhard/SwissTextSDG/checkpoints/checkpoint-710"

m = PeftModel.from_pretrained(model, adapter_checkpoint)
m = m.merge_and_unload()
tokenizer  = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token
model.config.pad_token_id = tokenizer.pad_token_id

/mnt/home/veralarabernhard/SwissTextSDG/sdg_env/lib/python3.8/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [24]:
text = "This is an abstract about water filtering"
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [27]:
outputs = model(**inputs)


In [32]:
predicted_probabilities = outputs.logits.softmax(dim=1)
predicted_class = predicted_probabilities.argmax(dim=1).item()
print(predicted_class)

13


: 